In [ ]:
import pandas as pd

df = pd.read_csv('RanCat_predictions_JFMAMJ(Final)_segment_reasons.csv')

# Reason categories
reason_buckets = {
    "Policy Type Related": [
        "Policy having TY Onwards Renewal Type",
        "Roll Over Business Type",
        "Tie Up with HYUNDAI",
        "Tie Up with Non-OEM",
        "Tie Up with MIBL OEM",
        "Minimal Policies Purchased"
    ],
    "Vehicle Age Related": [
        "Young Vehicle Age",
        "Old Vehicle Age"
    ],
    "Premium Related": [
        "High Own-Damage Premium",
        "High Total Premium Payable",
        "High Add-On Premium",
        "High Third-Party Premium",
        "High Discount with NCB",
        "Low Discount with NCB",
        "Higher Renewal Premium Impact",
        "Renewal Rate No Change & High Total Premium Payable",
        "Low Vehicle IDV",
        "Low No Claim Bonus Percentage",
        "High Vehicle IDV & High Total Premium",
        "Claims Happened"
    ],
    "Customer Related": [
        "Minimal Customer Tenure"
    ],
    "Organic Churn": [
        "Organic Churn"
    ]
}

# Function to classify reason buckets
def classify_reason(row):
    # Handle null or empty rows
    if pd.isnull(row) or row.strip() == "":
        return None
    
    # Initialize an empty set for categories
    categories = set()
    
    # Split reasons and check each reason's category
    reasons = row.split(", ")
    for reason in reasons:
        for category, keywords in reason_buckets.items():
            if reason in keywords:
                categories.add(category)
    
    # Sort categories alphabetically
    sorted_categories = sorted(categories)
    
    # Format output with commas and "and"
    if len(sorted_categories) > 1:
        return ", ".join(sorted_categories[:-1]) + " and " + sorted_categories[-1] + " issues"
    elif sorted_categories:
        return sorted_categories[0] + " issues"
    else:
        return None  

# Ensure 'Not Renewed Reasons' column is treated as string and apply the function
df['Not Renewed Reasons'] = df['Not Renewed Reasons'].astype(str)
df['Reason Buckets'] = df['Not Renewed Reasons'].apply(classify_reason)

df.to_csv('RanCat_predictions_JFMAMJ(Final)_segment_reasons_buckets.csv', index=False)

In [ ]:
import pandas as pd
import re
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

query = 'SELECT * FROM public.overall_cleaned_base_and_pr_ef_policyef_with_reasons;'
df = pd.read_sql(query, con=engine)

In [ ]:
# Reason categories
reason_buckets = {
    "Policy Type Related": [
        "Policy having TY Onwards Renewal Type",
        "Roll Over Business Type",
        "Tie Up with HYUNDAI",
        "Tie Up with Non-OEM",
        "Tie Up with MIBL OEM",
        "Minimal Policies Purchased"
    ],
    "Vehicle Age Related": [
        "Young Vehicle Age",
        "Old Vehicle Age"
    ],
    "Premium Related": [
        "High Own-Damage Premium",
        "High Total Premium Payable",
        "High Add-On Premium",
        "High Third-Party Premium",
        "High Discount with NCB",
        "Low Discount with NCB",
        "Higher Renewal Premium Impact",
        "Renewal Rate No Change & High Total Premium Payable",
        "Low Vehicle IDV",
        "Low No Claim Bonus Percentage",
        "High Vehicle IDV & High Total Premium",
        "Claims Happened"
    ],
    "Customer Related": [
        "Minimal Customer Tenure"
    ],
    "Organic Churn": [
        "Organic Churn"
    ]
}

# Function to classify reason buckets
def classify_reason(row):
    # Handle null or empty rows
    if pd.isnull(row) or row.strip() == "":
        return None
    
    # Initialize an empty set for categories
    categories = set()
    
    # Split reasons and check each reason's category
    reasons = row.split(", ")
    for reason in reasons:
        for category, keywords in reason_buckets.items():
            if reason in keywords:
                categories.add(category)
    
    # Sort categories alphabetically
    sorted_categories = sorted(categories)
    
    # Format output with commas and "and"
    if len(sorted_categories) > 1:
        return ", ".join(sorted_categories[:-1]) + " and " + sorted_categories[-1] + " issues"
    elif sorted_categories:
        return sorted_categories[0] + " issues"
    else:
        return None  

# Ensure 'Not Renewed Reasons' column is treated as string and apply the function
df['Not Renewed Reasons'] = df['Not Renewed Reasons'].astype(str)
df['Reason Buckets'] = df['Not Renewed Reasons'].apply(classify_reason)

# Export the results to a PostgreSQL table
output_table_name = "overall_cleaned_base_and_pr_ef_policyef_with_reasons_bucket"
df.to_sql(output_table_name, con=engine, if_exists='replace', index=False)

print(f"Data successfully saved to table '{output_table_name}' in the database.")

Data successfully saved to table 'overall_cleaned_base_and_pr_ef_policyef_with_reasons_bucket' in the database.
